In [1]:
import pandas as pd
import numpy as np

In [37]:
# Directamente cargo los archivos para no tener que correr todo el código anterior.
data=pd.read_csv("csv_limpios/data.csv")
df_belongs_to_collection=pd.read_csv("csv_limpios/collection.csv")
df_production_countries=pd.read_csv("csv_limpios/countries.csv")
df_production_companies=pd.read_csv("csv_limpios/companies.csv")
df_crew=pd.read_csv("csv_limpios/crew.csv")
df_genres=pd.read_csv("csv_limpios/genres.csv")
df_cast=pd.read_csv("csv_limpios/cast.csv")

# Preparar los df
Hay que ponerlos de forma que se puedan juntar todos a uno (unirlos por id_pelicula, y que cada columna importante tenga su id -cambiar el nombre a aquellos que ya tengan id, para que no se repita el mismo nombre de -) y de ahi decidir las columnas que considero innecesarias para sacarlas.
Más adelante se va a usar los df sin unir para conseguir la información necesaria desde el df que junta 

### data

In [38]:
data.head(1)

,id_pelicula,title,overview,status,runtime,popularity,tagline,original_language,vote_average,vote_count,release_date,release_year,budget,revenue,return
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Released,81.0,21.946943,NaN,en,7.7,5415.0,1995-10-30,1995,30000000.0,373554033.0,12.451801


Considero innecesaria la fecha completa de estreno, voy a dejar solo el año.

In [39]:
data.drop(columns='release_date',inplace=True)

In [40]:
data.head(1)

,id_pelicula,title,overview,status,runtime,popularity,tagline,original_language,vote_average,vote_count,release_year,budget,revenue,return
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Released,81.0,21.946943,NaN,en,7.7,5415.0,1995,30000000.0,373554033.0,12.451801


In [41]:
type(data['release_year'][0])

numpy.int64

Uno los dataframe a uno grande para realizar el analisis de relaciones entre las variables. Hay df que considere innecesarios de unir (la colección de peliculas, por ej)

Cambio los nombres de los id de cada df para diferenciarlos y que no se superpongan, al cambiarlos voy a poner "df_id" o una abreviación del nombre del df

### countries

In [42]:
df_production_countries.head(1)

,iso_3166_1,name,id_pelicula
0,US,United States of America,862


Como los paises no tienen id les creo unos con el metodo 'factorize' de pandas

In [43]:
a=pd.factorize(df_production_countries['name'])[0]

In [44]:
df_production_countries['countrie_id']=a

In [45]:
df_production_countries

,iso_3166_1,name,id_pelicula,countrie_id
0,US,United States of America,862,0
1,US,United States of America,8844,0
2,US,United States of America,15602,0
3,US,United States of America,31357,0
4,US,United States of America,11862,0
...,...,...,...,...
49403,US,United States of America,30840,0
49404,PH,Philippines,111109,47
49405,US,United States of America,67758,0
49406,RU,Russia,227506,20


In [46]:
df_production_countries[df_production_countries['countrie_id']==100].head(3)

,iso_3166_1,name,id_pelicula,countrie_id
11481,MC,Monaco,28055,100
18296,MC,Monaco,36970,100
19469,MC,Monaco,41211,100


In [47]:
df_production_countries['name_countrie']=df_production_countries['name']
df_production_countries.drop(columns='name',inplace=True)
df_production_countries['short_name_countrie']=df_production_countries['iso_3166_1']
df_production_countries.drop(columns='iso_3166_1',inplace=True)

### companies

In [60]:
df_production_companies.head(5)

,name,id_pelicula,revenue,companie_id
0,Pixar Animation Studios,862,373554033.0,3.0
1,TriStar Pictures,8844,262797249.0,559.0
2,Teitler Film,8844,262797249.0,2550.0
3,Interscope Communications,8844,262797249.0,10201.0
4,Warner Bros.,15602,0.0,6194.0


In [ ]:
df_production_companies[df_production_companies['id']==3.0]

In [48]:
df_production_companies['companie_id']=df_production_companies['id']
df_production_companies.drop(columns='id',inplace=True)

In [62]:
df_production_companies[df_production_companies['companie_id']==10201]

,name,id_pelicula,revenue,companie_id
3,Interscope Communications,8844,262797249.0,10201.0
159,Interscope Communications,2054,106269971.0,10201.0
451,Interscope Communications,79593,5780000.0,10201.0
671,Interscope Communications,161495,12400000.0,10201.0
884,Interscope Communications,46094,21011500.0,10201.0
1167,Interscope Communications,9057,16478900.0,10201.0
1288,Interscope Communications,46029,0.0,10201.0
1407,Interscope Communications,27281,0.0,10201.0
1434,Interscope Communications,43634,0.0,10201.0
1650,Interscope Communications,11511,0.0,10201.0


### genres

In [63]:
df_genres.head(5)

,name,id_pelicula,genre_id
0,Animation,862,16.0
1,Comedy,862,35.0
2,Family,862,10751.0
3,Adventure,8844,12.0
4,Fantasy,8844,14.0


In [49]:
df_genres['genre_id']=df_genres['id']
df_genres.drop(columns='id',inplace=True)

In [ ]:
df_genres.head(1)

In [64]:
df_genres[df_genres['genre_id']==16]

,name,id_pelicula,genre_id
0,Animation,862,16.0
34,Animation,21032,16.0
124,Animation,10530,16.0
557,Animation,15789,16.0
572,Animation,43475,16.0
...,...,...,...
90837,Animation,460135,16.0
90867,Animation,175457,16.0
90868,Animation,184402,16.0
90958,Animation,455661,16.0


### crew

In [65]:
df_crew.head(5)

,department,gender,job,name,id_pelicula,crew_id
0,Directing,2.0,Director,John Lasseter,862,7879.0
1,Writing,2.0,Screenplay,Joss Whedon,862,12891.0
2,Writing,2.0,Screenplay,Andrew Stanton,862,7.0
3,Writing,2.0,Screenplay,Joel Cohen,862,12892.0
4,Writing,0.0,Screenplay,Alec Sokolow,862,12893.0


In [50]:
df_crew['crew_id']=df_crew['id']
df_crew.drop(columns='id',inplace=True)

In [66]:
df_crew.head(1)

,department,gender,job,name,id_pelicula,crew_id
0,Directing,2.0,Director,John Lasseter,862,7879.0


In [67]:
df_crew[df_crew['crew_id']==12893]

,department,gender,job,name,id_pelicula,crew_id
4,Writing,0.0,Screenplay,Alec Sokolow,862,12893.0
24648,Writing,0.0,Screenplay,Alec Sokolow,9416,12893.0
40236,Writing,0.0,Screenplay,Alec Sokolow,49981,12893.0
107520,Writing,0.0,Screenplay,Alec Sokolow,11007,12893.0
119168,Writing,0.0,Screenplay,Alec Sokolow,8920,12893.0
161082,Writing,0.0,Screenplay,Alec Sokolow,9513,12893.0
457621,Production,0.0,Producer,Alec Sokolow,430448,12893.0


### Cast

En el caso de cast ya contaba con cast_id pero se encontraban repetidos en gran proporción y no parecen dar información importante en el df_cast, parece más un id que diferencia el cast dentro de cada pelicula solamente, haciendo que los id se repitan ya que para cada pelicula este id se reinicia, por lo tanto no nos sirve.

En su lugar vamos a utilizar los valores de la columna id que se repiten solamente por la repetición de trabajos de una misma persona, es decir, este id pertenece a una persona en particular(es único).

In [51]:
df_cast

,cast_id,character,gender,id,name,order,id_pelicula
0,14.0,Woody (voice),2.0,31.0,Tom Hanks,0.0,862
1,15.0,Buzz Lightyear (voice),2.0,12898.0,Tim Allen,1.0,862
2,16.0,Mr. Potato Head (voice),2.0,7167.0,Don Rickles,2.0,862
3,17.0,Slinky Dog (voice),2.0,12899.0,Jim Varney,3.0,862
4,18.0,Rex (voice),2.0,12900.0,Wallace Shawn,4.0,862
...,...,...,...,...,...,...,...
562469,2.0,NaN,2.0,544742.0,Iwan Mosschuchin,0.0,227506
562470,3.0,NaN,1.0,1090923.0,Nathalie Lissenko,1.0,227506
562471,4.0,NaN,2.0,1136422.0,Pavel Pavlov,2.0,227506
562472,5.0,NaN,0.0,1261758.0,Aleksandr Chabrov,3.0,227506


In [52]:
df_cast['cast_id'].value_counts()

4.0       26777
5.0       25655
3.0       25594
6.0       24006
2.0       22846
          ...  
344.0         1
1099.0        1
1098.0        1
1097.0        1
438.0         1
Name: cast_id, Length: 574, dtype: int64

In [53]:
df_cast['id'].value_counts()

121323.0     241
113.0        148
4165.0       125
2231.0       123
16927.0      110
            ... 
1748750.0      1
1088588.0      1
1722007.0      1
28519.0        1
1261758.0      1
Name: id, Length: 206158, dtype: int64

In [54]:
df_cast[df_cast['cast_id']==4.0].head(3)

,cast_id,character,gender,id,name,order,id_pelicula
41,4.0,Ariel Gustafson,1.0,13567.0,Ann-Margret,2.0,15602
49,4.0,Robin Stokes,1.0,51359.0,Lela Rochon,3.0,31357
59,4.0,Annie Banks-MacKenzie,1.0,70696.0,Kimberly Williams-Paisley,3.0,11862


In [55]:
df_cast[df_cast['id']==121323.0].head(3)

,cast_id,character,gender,id,name,order,id_pelicula
13755,36.0,Audience Member (uncredited),1.0,121323.0,Bess Flowers,9.0,872
13784,10.0,Fashion Show Spectator,1.0,121323.0,Bess Flowers,5.0,13320
13817,24.0,Diner at Ernie's,1.0,121323.0,Bess Flowers,9.0,426


In [56]:
df_cast['cast_id']=df_cast['id']
df_cast.drop(columns='id',inplace=True)

### collection

In [59]:
df_belongs_to_collection.head(5)

,id,name,id_pelicula,revenue,budget
0,10194,Toy Story Collection,862,373554033.0,30000000.0
1,119050,Grumpy Old Men Collection,15602,0.0,0.0
2,96871,Father of the Bride Collection,11862,76578911.0,0.0
3,645,James Bond Collection,710,352194034.0,58000000.0
4,117693,Balto Collection,21032,11348324.0,0.0


In [70]:
df_belongs_to_collection[df_belongs_to_collection['id']==645].head(5)

,id,name,id_pelicula,revenue,budget
3,645,James Bond Collection,710,352194034.0,58000000.0
236,645,James Bond Collection,714,333011068.0,110000000.0
361,645,James Bond Collection,707,152427960.0,30000000.0
482,645,James Bond Collection,658,124881062.0,2500000.0
483,645,James Bond Collection,657,78898765.0,2000000.0


In [71]:
# Renombro la columna 'name' para distinguirla de otras
df_belongs_to_collection['collection_name']=df_belongs_to_collection['name']
df_belongs_to_collection['collection_id']=df_belongs_to_collection['id']

# Elimino las columnas que duplique en su momento al crear este df, por si acaso.
df_belongs_to_collection.drop(columns=['name','id','revenue','budget'],inplace=True)

In [75]:
df_belongs_to_collection.shape

(4488, 3)

In [74]:
df_belongs_to_collection.head(5)

,id_pelicula,collection_name,collection_id
0,862,Toy Story Collection,10194
1,15602,Grumpy Old Men Collection,119050
2,11862,Father of the Bride Collection,96871
3,710,James Bond Collection,645
4,21032,Balto Collection,117693


In [73]:
df_belongs_to_collection[df_belongs_to_collection['collection_id']==645].head(5)

,id_pelicula,collection_name,collection_id
3,710,James Bond Collection,645
236,714,James Bond Collection,645
361,707,James Bond Collection,645
482,658,James Bond Collection,645
483,657,James Bond Collection,645


## Merge

Este merge es principalmente para ver la tabla de correlaciones de las variables.<br>
A la hora de utilizar parametros la idea es solamente usar los id para predecir (evita errores el usar numeros que identifiquen cada categoria, y también resultan ser dos columnas que dan la misma información la que tienen el nombre con las que tienen el id del nombre).<br>
Luego como mencioné más arriba voy a usar esos id una vez predichos para compararlos con el df que contiene todos sus datos y sacar el dato de nombre de ahi.

In [ ]:
df_todo=pd.merge(data,df_genres,on='id_pelicula')
df_todo=pd.merge(df_todo,df_crew,on='id_pelicula')
df_todo=pd.merge(df_todo,df_cast,on='id_pelicula')
df_todo=pd.merge(df_todo,df_production_companies,on='id_pelicula')
df_todo=pd.merge(df_todo,df_production_countries,on='id_pelicula')
df_todo=pd.merge(df_todo,df_belongs_to_collection,on='id_pelicula')